*Copyright (c) Microsoft Corporation. All rights reserved.*                 
*Licensed under the MIT License.*

In [ ]:
%matplotlib inline

# Interpreting Bag of Words Text Classification models

This notebook show a set by step implementation of traditional machine algorithms with a bag of words representation as an interpretable model using feature importances.

###### Note:
* This example walks through a logistic regression baseline with a simple Bag of Words encoding. However, any model that follows sklearn's classifier API should be supported natively or with minimal tweaking.
* The interpreter supports interpretations using either coefficients associated with linear models or feature importances associated with ensemble models.
* The classifier relies on scipy's sparse representations to keep the dataset in memory.

In [ ]:
#import libraries
import sys
import os
import json
import pandas as pd
import numpy as np
import scipy
import string
from scrapbook.api import glue
working_dir = %pwd

from interpret_text.bow.BOWExplainer import BOWExplainer

# sklearn
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# spacy and nlp recipes
import spacy
from spacy.lang.en import English
from utils_nlp.dataset.multinli import load_pandas_df

## Introduction

The notebook is built on features made available by [Sci-kit Learn](https://scikit-learn.org/stable/) and [spacy](https://spacy.io/) for easier compatibiltiy with popular tookits.                     
The notebook also relies on the pip installable "utils_nlp.dataset.multinli" package from Microsoft's [NLP-recipes](https://github.com/microsoft/nlp-recipes/tree/master/utils_nlp) open source package for dataloading purposes.               


### An Overview

The [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) allows a 1:1 mapping from the individual words to their respective frequencies in the Document-term matrix.                                
We use spacy's medium language [model](https://spacy.io/models/en#en_core_web_md) that's trained on common crawl for text preprocessing.

Set configuration parameters

In [ ]:
DATA_FOLDER = './temp'
TRAIN_SIZE = 0.7
TEST_SIZE = 0.3

Load data

In [ ]:
df = load_pandas_df(DATA_FOLDER, "train")
df = df[df["gold_label"]=="neutral"]  # get unique sentences

# fetch documents and labels from data frame
X_str = df['sentence1'] # the document we want to analyze
ylabels = df['genre'] # the labels, or answers, we want to test against

## Configure preprocessessing setup

In the default BOWExplainer, each text document in the dataset undergoes the following preprocessing steps during conversion into tokens:
* Use spacy's English parser to parse document.
* Convert to lower case.
* Strip off white space surrounding each word.
* [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) based on spacy's language model.
* Remove stop words and punctuation to obtain tokens.

## Preprocess training data and labels and cast into correct format
* Count tokens and construct sparse term document matrix.
* Vectorize tokens into counts.
* Convert labels from strings into integers.

###### Note: Vocabulary

* *The vocabulary is compiled from the dataset. This means that any word that does not appear in the Dataset is not a part of the vocabulary. This also means that the vocabulary varies with the dataset.*           
* *Every word that appear one or more times is considered to be part of the vocab.*
* *However, The sklearn countvectorizer allows addition of a custom vocabulary as an input parameter.*

## Configure training setup

* Split data into train and test using a random shuffle
* load desired classifier. In this case, Logistic Regression.
* Setup grid search for hyperparameter optimization and train model. Edit the hyper parameter range to search over as per your model.
* Fit models to train set

In [ ]:
# Create explainer object that contains defaault glassbox classfier and explanation methods
bow_explainer = BOWExplainer()
# encode document and labels into vector form using 1-gram Bag of Words.
X_train, X_test, y_train, y_test = bow_explainer.encode(X_str,ylabels)

In [ ]:
print("X_train shape =" + str(X_train.shape))
print("y_train shape =" + str(y_train.shape))
print("X_train data structure = "+ str(type(X_train)))

###### Note:
* *Set the parameters using cross-validation*
* *Below listed hyperparamters are selected by searching over a larger space.*
* *These apply speciically to this instance of the logistic regression model and mnli dataset.*
* *'Multinomial' setup was found to be better than 'one-vs-all' across the board*
* *Default 'liblinear' solver is not supported for 'multinomial' model setup*
* *For a different model or dataset, set the range as appropriate using the hyperparam_range argument in the train method* 

In [ ]:
classifier, best_params = bow_explainer.train(X_train,y_train)

## Results
* Obtain best model and corresponding hyper parameters
* Report Accuracy, F1 score, Precision and Recall.

In [ ]:
# obtain best classifier and hyper params
print("best classifier: " + str(best_params))

Report performance metrics

In [ ]:
mean_accuracy = classifier.score(X_test, y_test, sample_weight=None)
print("accuracy = " + str(mean_accuracy*100) + "%")
y_pred = classifier.predict(X_test)
[precision, recall, fscore, support] = precision_recall_fscore_support(y_test, y_pred,average='macro')

Capture metrics for integration testing

In [ ]:
glue("accuracy", mean_accuracy)
glue("precision", precision)
glue("recall", recall)
glue("f1", fscore)
print ("[precision, recall, fscore, support] = " + str([precision, recall, fscore, support]))

## Compute global importances

Visualize the most and least important words for each class over the entire dataset
Choose which label's importances to visualize 

In [ ]:
print("The class names are as follows")
print(bow_explainer.preprocessor.labelEncoder.classes_)
label_name = "fiction"

#Obtain the top feature ids for the selected class label.           
#Map top features back to words.
top_words, top_importances = bow_explainer.explain_global(label_name)

## Compute local importances

In [ ]:
#Enter any document & label pair that needs to be interpreted
document = "I travelled to the beach. I took the train. I saw faries, dragons and elves"

#Obtain the top feature ids for the selected class label
word_importances, parsed_sentence = bow_explainer.explain_local(document, model=None)

In [ ]:
parsed_sentence

In [ ]:
#Visualize local feature importances as a heatmap over words in the document
bow_explainer.visualize(word_importances, parsed_sentence)